In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import pickle
data = pickle.load(open('IMDB/imdb_full.pkl', 'rb'))

import json
vocab = json.load(open('IMDB/imdb_word_index.json'))

In [5]:
inv = {idx:word for word, idx in vocab.items()}

In [6]:
(X, y), (Xt, yt) = data

In [7]:
trainidx = [i for i, x in enumerate(X) if len(x) < 400]
X = [X[i] for i in trainidx]
y = [y[i] for i in trainidx]

testidx = [i for i, x in enumerate(Xt) if len(x) < 400]
testidx, remaining_idx =  train_test_split(testidx, train_size=0.2, random_state=1378)
Xr = [Xt[i] for i in remaining_idx]
yr = [yt[i] for i in remaining_idx]

Xt = [Xt[i] for i in testidx]
yt = [yt[i] for i in testidx]

In [8]:
X = [[inv[x] for x in doc] for doc in X]
Xt = [[inv[x] for x in doc] for doc in Xt]
Xr = [[inv[x] for x in doc] for doc in Xr]

In [9]:
X = [" ".join(x) for x in X]
Xt = [" ".join(x) for x in Xt]
Xr = [" ".join(x) for x in Xr]

In [10]:
import vectorizer

In [11]:
vec = vectorizer.Vectorizer(min_df=20)

In [12]:
vec.fit(X + Xt + Xr)

In [13]:
vec.seq_text = {}
vec.seq_text['train'] = vec.get_seq_for_docs(X)
vec.seq_text['test'] = vec.get_seq_for_docs(Xt)
vec.seq_text['rem'] = vec.get_seq_for_docs(Xr)

In [14]:
vec.label = { 'train' : y, 'test' : yt , 'rem' : yr} 

In [15]:
from torchtext.vocab import Vectors, GloVe, CharNGram, FastText
url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
vectors = Vectors('wiki.simple.vec', url=url)

In [16]:
vec.word_dim = vectors.dim

In [17]:
vec.embeddings = np.zeros((len(vec.idx2word), vec.word_dim))
for i, word in vec.idx2word.items() :
    vec.embeddings[i] = vectors[word].numpy()

In [18]:
pickle.dump(vec, open('IMDB/imdb_data.p', 'wb'))